In [2]:
from keras.datasets import boston_housing
from keras import models, layers
import numpy as np

# Data Acquisition

In [3]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [4]:
train_data.shape

(404, 13)

In [5]:
train_data

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [6]:
test_data.shape

(102, 13)

In [7]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

# Data Preparation

In [8]:
# feature-wise normalization
mean = train_data.mean(axis=0)
train_data -= mean

std = train_data.std(axis=0)
train_data /= std

# Note: quantity used is computed from train data. u should never use  quantity computed on the test data
test_data -= mean
test_data /= std

train_data.shape[0]

404

# Building a model

In [17]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

# Validation using K-fold validation

In [23]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 500
all_mae_histories = []

for i in range(k):
    #     Prepare partition
    print('Processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) *num_val_samples ]    
    val_targets = train_targets[i * num_val_samples: (i + 1) *num_val_samples ]
    
    #     Partial Training Data
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)
    
    #     Building a model
    model = build_model()
    

    #     Trains the model in silent model (verbose = 0)
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
average_mae_history
    
#     #     Evaluate models on validation data
#     val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
#     all_scores.append(val_mae)



Processing fold # 0
Epoch 1/500
303/303 [==============================] - 1s 3ms/step - loss: 178.1278 - mae: 9.7632A: 0s - loss: 299.0
Epoch 2/500
303/303 [==============================] - 1s 3ms/step - loss: 30.2464 - mae: 3.7901
Epoch 3/500
303/303 [==============================] - 1s 3ms/step - loss: 22.9310 - mae: 3.2861
Epoch 4/500
303/303 [==============================] - 1s 3ms/step - loss: 19.4649 - mae: 2.8721
Epoch 5/500
303/303 [==============================] - 1s 3ms/step - loss: 17.6693 - mae: 2.7782
Epoch 6/500
303/303 [==============================] - 1s 3ms/step - loss: 16.3633 - mae: 2.6133
Epoch 7/500
303/303 [==============================] - 1s 3ms/step - loss: 15.1994 - mae: 2.6002
Epoch 8/500
303/303 [==============================] - 1s 3ms/step - loss: 13.6789 - mae: 2.4147
Epoch 9/500
303/303 [==============================] - 1s 3ms/step - loss: 13.5396 - mae: 2.3854
Epoch 10/500
303/303 [==============================] - 1s 3ms/step - loss: 13.2236 - m

303/303 [==============================] - 1s 3ms/step - loss: 4.4930 - mae: 1.4364
Epoch 85/500
303/303 [==============================] - 1s 3ms/step - loss: 4.7450 - mae: 1.4571
Epoch 86/500
303/303 [==============================] - 1s 3ms/step - loss: 4.6131 - mae: 1.4322
Epoch 87/500
303/303 [==============================] - 1s 3ms/step - loss: 4.8273 - mae: 1.4924
Epoch 88/500
303/303 [==============================] - 1s 3ms/step - loss: 4.8440 - mae: 1.4191
Epoch 89/500
303/303 [==============================] - 1s 3ms/step - loss: 4.8531 - mae: 1.4205
Epoch 90/500
303/303 [==============================] - 1s 3ms/step - loss: 4.6948 - mae: 1.3917
Epoch 91/500
303/303 [==============================] - 1s 3ms/step - loss: 4.6462 - mae: 1.3830
Epoch 92/500
303/303 [==============================] - 1s 4ms/step - loss: 4.9496 - mae: 1.4805
Epoch 93/500
303/303 [==============================] - 1s 3ms/step - loss: 4.3834 - mae: 1.3567
Epoch 94/500
303/303 [=====================

303/303 [==============================] - 1s 3ms/step - loss: 2.6845 - mae: 1.0736
Epoch 168/500
303/303 [==============================] - 1s 4ms/step - loss: 2.3827 - mae: 1.0967
Epoch 169/500
303/303 [==============================] - 1s 3ms/step - loss: 3.0057 - mae: 1.1448
Epoch 170/500
303/303 [==============================] - 1s 3ms/step - loss: 2.4724 - mae: 1.0900
Epoch 171/500
303/303 [==============================] - 1s 3ms/step - loss: 2.3662 - mae: 1.0690
Epoch 172/500
303/303 [==============================] - 1s 3ms/step - loss: 2.5702 - mae: 1.1180
Epoch 173/500
303/303 [==============================] - 1s 4ms/step - loss: 2.6272 - mae: 1.1138
Epoch 174/500
303/303 [==============================] - 1s 3ms/step - loss: 2.5647 - mae: 1.1576
Epoch 175/500
303/303 [==============================] - 1s 3ms/step - loss: 2.5486 - mae: 1.1216
Epoch 176/500
303/303 [==============================] - 1s 3ms/step - loss: 2.7143 - mae: 1.1158
Epoch 177/500
303/303 [===========

303/303 [==============================] - 1s 3ms/step - loss: 1.5651 - mae: 0.8919
Epoch 251/500
303/303 [==============================] - 1s 3ms/step - loss: 1.5356 - mae: 0.9163
Epoch 252/500
303/303 [==============================] - 1s 3ms/step - loss: 1.7859 - mae: 0.9684
Epoch 253/500
303/303 [==============================] - 1s 3ms/step - loss: 1.7730 - mae: 0.9226
Epoch 254/500
303/303 [==============================] - 1s 3ms/step - loss: 1.8430 - mae: 0.9339
Epoch 255/500
303/303 [==============================] - 1s 3ms/step - loss: 1.7758 - mae: 0.9578
Epoch 256/500
303/303 [==============================] - 1s 3ms/step - loss: 1.6142 - mae: 0.9262
Epoch 257/500
303/303 [==============================] - 1s 3ms/step - loss: 1.9508 - mae: 0.9951
Epoch 258/500
303/303 [==============================] - 1s 3ms/step - loss: 1.7191 - mae: 0.9203
Epoch 259/500
303/303 [==============================] - 1s 4ms/step - loss: 1.8598 - mae: 1.0307
Epoch 260/500
303/303 [===========

303/303 [==============================] - 1s 3ms/step - loss: 1.2822 - mae: 0.8191
Epoch 334/500
303/303 [==============================] - 1s 3ms/step - loss: 1.4260 - mae: 0.8400
Epoch 335/500
303/303 [==============================] - 1s 3ms/step - loss: 1.3194 - mae: 0.7840
Epoch 336/500
303/303 [==============================] - 1s 3ms/step - loss: 1.1781 - mae: 0.8054
Epoch 337/500
303/303 [==============================] - 1s 3ms/step - loss: 1.2988 - mae: 0.7712
Epoch 338/500
303/303 [==============================] - 1s 3ms/step - loss: 1.1739 - mae: 0.8010
Epoch 339/500
303/303 [==============================] - 1s 3ms/step - loss: 1.0885 - mae: 0.7457
Epoch 340/500
303/303 [==============================] - 1s 3ms/step - loss: 1.2282 - mae: 0.7872
Epoch 341/500
303/303 [==============================] - 1s 3ms/step - loss: 1.1583 - mae: 0.7789
Epoch 342/500
303/303 [==============================] - 1s 3ms/step - loss: 1.3117 - mae: 0.7785
Epoch 343/500
303/303 [===========

303/303 [==============================] - 2s 5ms/step - loss: 1.0519 - mae: 0.7333
Epoch 417/500
303/303 [==============================] - 2s 6ms/step - loss: 1.1892 - mae: 0.7726
Epoch 418/500
303/303 [==============================] - 2s 7ms/step - loss: 0.9745 - mae: 0.7365
Epoch 419/500
303/303 [==============================] - 3s 10ms/step - loss: 1.0182 - mae: 0.7090
Epoch 420/500
303/303 [==============================] - 2s 7ms/step - loss: 0.9303 - mae: 0.6964
Epoch 421/500
303/303 [==============================] - 2s 7ms/step - loss: 0.8974 - mae: 0.7115
Epoch 422/500
303/303 [==============================] - 2s 7ms/step - loss: 1.0319 - mae: 0.7112
Epoch 423/500
303/303 [==============================] - 2s 7ms/step - loss: 1.0433 - mae: 0.7148
Epoch 424/500
303/303 [==============================] - 2s 7ms/step - loss: 1.0231 - mae: 0.7258
Epoch 425/500
303/303 [==============================] - 2s 7ms/step - loss: 0.9842 - mae: 0.7117
Epoch 426/500
303/303 [==========

303/303 [==============================] - 2s 8ms/step - loss: 0.8117 - mae: 0.6696
Epoch 500/500
303/303 [==============================] - 2s 6ms/step - loss: 0.8880 - mae: 0.6746


KeyError: 'val_mean_absolute_error'